In [1]:
import json
from pathlib import Path

import glob

import pickle
from coffea import processor


import numpy as np
import pandas as pd


import hist
import matplotlib.pyplot as plt


import pandas as pd

In [65]:
import pickle

with open('outfiles/tt_CR/ttbar_h/2023-09-14/2017/TTTo2L2Nu_1.pkl', 'rb') as file:
    data = pickle.load(file)
    
data['histograms']['top_mass'].project("top_mrec")#['tau_kin'].project("tau_phi")

Hist(Variable([120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220], name='top_mrec'), storage=Weight()) # Sum: WeightedSum(value=6687.01, variance=436200) (WeightedSum(value=540630, variance=3.7026e+07) with flow)

In [45]:
# load cross sections
with open(f"wprime_plus_b/data/DAS_xsec.json", "r") as f:
    xsecs = json.load(f)
    
# define luminosity
lumi = 4147.7877399

output_directory = f"outfiles/tt_CR/ttbar_h/2023-09-14/2017/"

In [7]:
def group_outputs(output_directory: str) -> dict:
    """group output .pkl files by sample"""
    output_files = glob.glob(f"{output_directory}/*.pkl", recursive=True)
    grouped_outputs = {}
    for output_file in output_files:
        # get output file names
        sample_name = output_file.split("/")[-1].split(".pkl")[0]
        if sample_name.rsplit("_")[-1].isdigit():
            sample_name = "_".join(sample_name.rsplit("_")[:-1])
        # append file names to grouped_outputs
        if sample_name in grouped_outputs:
            grouped_outputs[sample_name].append(output_file)
        else:
            grouped_outputs[sample_name] = [output_file]
            
    return grouped_outputs

grouped_outputs = group_outputs(output_directory)

#print(grouped_outputs)

In [8]:
def open_output(output_fname: str) -> dict:
    """open .pkl output file"""
    with open(output_fname, "rb") as f:
        output = pickle.load(f)
    return output

def accumulate_outputs(grouped_outputs: dict) -> dict:
    """accumulate output arrays by sample"""
    accumulated_outputs = {}
    for sample in grouped_outputs:
        accumulated_outputs[sample] = []
        for output_fname in grouped_outputs[sample]:
            output = open_output(output_fname)
            accumulated_outputs[sample].append(output)
        accumulated_outputs[sample] = processor.accumulate(accumulated_outputs[sample])
    return accumulated_outputs

accumulated_outputs = accumulate_outputs(grouped_outputs)
#print(accumulated_outputs)

In [38]:
mcs_dataset_TT = {"TTToHadronic": [], "TTTo2L2Nu": [], "TTToSemiLeptonic": []}
data_dataset = {"Tau": []}


def build_report(accumulated_outputs: dict, mcs_dataset_TT, mcs_dataset_DY, mcs_dataset_WJ, mcs_dataset_ST, mcs_dataset_VV):
    mcs = ["DYJetsToLL", "WJetsToLNu", "VV", "tt", "SingleTop", "Higgs"]
    events = {sample: 0 for sample in mcs}
    events.update({"Data": 0})
    errors = events.copy()
    
    print(accumulated_outputs['SingleTau']['histograms']['tau_kin'])
    for key1, sub_dict in accumulated_outputs.items():
        for key2, sub_sub_dict in sub_dict.items():
            if ("TTTo" in key2):
                if("TTToHadronic" in key2):
                    mcs_dataset_TT['TTToHadronic']['histograms'].append(sub_sub_dict[histo_defined])
                elif("TTTo2L2Nu" in key2):
                    mcs_dataset_TT['TTTo2L2Nu']['histograms'].append(sub_sub_dict[histo_defined])  
                elif("TTToSemiLeptonic" in key2):
                    mcs_dataset_TT['TTToSemiLeptonic']['histograms'].append(sub_sub_dict[histo_defined]) 
                
            if ("SingleTau" in key2):
                data_dataset["SingleTau"]['histograms'].append(sub_sub_dict[histo_defined])
                

histo_defined = "tau_pt"
build_report(accumulated_outputs, mcs_dataset_TT, mcs_dataset_DY, mcs_dataset_WJ, mcs_dataset_ST, mcs_dataset_VV)

Hist(
  StrCategory(['SingleTau_3', 'SingleTau_5', 'SingleTau_1', 'SingleTau_4', 'SingleTau_2'], growth=True, name='dataset'),
  Variable(array([ 20.,  40.,  60.,  80., 100., 120., 140., 160., 180., 200., 240.,
       260., 280., 300., 320., 340., 360., 380., 400., 420., 440., 460.,
       480., 500.]), name='tau_pt'),
  Regular(24, -2.4, 2.4, name='tau_eta'),
  Regular(32, -3.14159, 3.14159, name='tau_phi'),
  StrCategory(['nominal'], growth=True, name='variation'),
  storage=Weight()) # Sum: WeightedSum(value=562, variance=562) (WeightedSum(value=564, variance=564) with flow)


In [44]:
accumulated_outputs['SingleTau']['histograms']['tau_kin']

Hist(
  StrCategory(['SingleTau_3', 'SingleTau_5', 'SingleTau_1', 'SingleTau_4', 'SingleTau_2'], growth=True, name='dataset'),
  Variable(array([ 20.,  40.,  60.,  80., 100., 120., 140., 160., 180., 200., 240.,
       260., 280., 300., 320., 340., 360., 380., 400., 420., 440., 460.,
       480., 500.]), name='tau_pt'),
  Regular(24, -2.4, 2.4, name='tau_eta'),
  Regular(32, -3.14159, 3.14159, name='tau_phi'),
  StrCategory(['nominal'], growth=True, name='variation'),
  storage=Weight()) # Sum: WeightedSum(value=562, variance=562) (WeightedSum(value=564, variance=564) with flow)

In [33]:
Histos_TTToHadronic = (mcs_dataset_TT['TTToHadronic'][0] + mcs_dataset_TT['TTToHadronic'][0] + mcs_dataset_TT['TTToHadronic'][0] + mcs_dataset_TT['TTToHadronic'][0]) * lumi * xsecs['TTToHadronic'][0]
Histos_TTTo2L2Nu = (mcs_dataset_TT['TTTo2L2Nu'][0] + mcs_dataset_TT['TTTo2L2Nu'][1] + mcs_dataset_TT['TTTo2L2Nu'][2] + mcs_dataset_TT['TTTo2L2Nu'][3]  + mcs_dataset_TT['TTTo2L2Nu'][4]) * lumi * xsecs['TTTo2L2Nu']
Histos_TTToSemiLeptonic = (mcs_dataset_TT['TTToSemiLeptonic'][0] + mcs_dataset_TT['TTToSemiLeptonic'][1]  + mcs_dataset_TT['TTToSemiLeptonic'][2]) * lumi * xsecs['TTToSemiLeptonic'] 


Histos_TTToHadronic.plot1d(label = "tt_Hadronic")
Histos_TTToSemiLeptonic.plot1d(label = "tt_SemiLeptonic")
Histos_TTTo2L2Nu.plot1d(label = "tt_2L2Nu")
plt.legend()
plt.axvline(173.1, color='r', linestyle='dashed', linewidth=1)
plt.title('Top tagger unresolve case "FatJet" (TT samples)')
plt.ylabel('Events')

IndexError: list index out of range

In [13]:
Hist_tt = Histos_TTToHadronic + Histos_TTTo2L2Nu + Histos_TTToSemiLeptonic

tt_events = Hist_tt.sum().value


Hist_tt.plot1d(label = "tt_full")
plt.legend()
plt.axvline(173.1, color='r', linestyle='dashed', linewidth=1)
plt.title('Top tagger (TT full dataset)')
plt.ylabel('Events')

NameError: name 'Histos_TTToHadronic' is not defined

In [14]:
# Data
Hist_data = data_dataset["Tau"][0] + data_dataset["Tau"][1] + data_dataset["Tau"][2] + data_dataset["Tau"][3] + data_dataset["Tau"][4]

data_events = Hist_data.sum().value


Hist_data.plot1d(label = "Data")
plt.legend()
plt.axvline(173.1, color='r', linestyle='dashed', linewidth=1)
plt.title('Top tagger (Data)')
plt.ylabel('Events')

IndexError: list index out of range

In [16]:
# Superposition
# Table
total_events_mc = tt_events + dy_events + wj_events + vv_events + st_events 

datos = {
    'Background': ['tt', 'Total Background', 'Data'],
    'Numer of events': [tt_events, total_events_mc, data_events],
    '%': [((tt_events/total_events_mc)*100), ((total_events_mc/total_events_mc)*100), '...']
}

df = pd.DataFrame(datos)

tabla_sin_indices = df.to_string(index=False)

print(tabla_sin_indices)



# Plot
Hist_tt.plot1d(label = "tt")
plt.legend()
plt.axvline(173.1, color='r', linestyle='dashed', linewidth=1)
plt.title('Top tagger')
plt.ylabel('Events')

NameError: name 'tt_events' is not defined